In [0]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [0]:
import pandas as pd
import urllib.request
import nltk
import datetime
from urllib.request import urlopen
from multiprocessing import Pool, cpu_count
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup

def remove_stopwords(tokens):
    for i, token in enumerate(tokens):
        tokens[i] = ''.join([ch for ch in token if ch not in char_blacklist])
    tokens_sw = [w.lower() for w in tokens if w not in stopwords]
    tokens_lemmatize = [wnl.lemmatize(token) for token in tokens_sw]
    return tokens_lemmatize


def crawl(object):
    print(f"{object['i']}/{len(urls)} || {object['url']}")
    tokens_lemmatize = ''
    try:
        req = urllib.request.Request(object['url'], headers=hdr)
        html = urlopen(req, timeout=15).read()
        soup = BeautifulSoup(html, "html.parser")
        [tag.decompose() for tag in soup("script")]
        [tag.decompose() for tag in soup("style")]
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk.lower() for chunk in chunks if chunk)
        # Tokenize text
        tokens = [token.lower() for token in word_tokenize(text)]
        # Remove stopwords
        tokens_lemmatize = remove_stopwords(tokens)
    except Exception:
        print(f"{object['i']}/{len(urls)} || {object['url']} FAILED. ")
    return tokens_lemmatize if len(tokens_lemmatize) else ''
    #     return page_tokens[object['i']]

date = '2019-10-23'
input_path = 'https://drive.google.com/uc?export=download&id=1bSHSVwBF7qrtUM2DXXqZXvc23qp9UW2B'
#output_path = f'Datasets/Feature_dataset_{date}.csv'

df = pd.read_excel(input_path,header = 0)
df.columns = ['id', 'url','target']
date = datetime.datetime.today().strftime("%Y-%m-%d")
df = df[160000:169900]
df.head()
hdr = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}
wnl = WordNetLemmatizer()
char_blacklist = list(
        chr(i) for i in range(32, 127) if (i <= 47 or i >= 58) and (i <= 64 or i >= 91) and (i <= 96 or i >= 123))
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(char_blacklist)

start = datetime.datetime.now()
print(df)

In [0]:
print(start)


In [0]:
urls = [{"i": index, "url": url} for index, url in enumerate(list(df['url'].values))]
print(urls)

In [0]:
df['tokens'] = ''
p = Pool(cpu_count() * 2)
tokens = p.map(crawl, urls)
df['tokens'][:len(tokens)] = tokens
stop = datetime.datetime.now()
print(stop)
exec_time = stop - start
print(exec_time)

In [0]:
df.head()

In [0]:
from google.colab import files
df.to_excel('df160k170k.xlsx',index=False)

In [0]:
from google.colab import files
files.download('df160k170k.xlsx')

In [0]:
# To Mount
from google.colab import drive
drive.mount('drive')
# To save on drive
!cp df160k170k.xlsx drive/My\ Drive/